# Building Agentic video RAG with Strands Agents and Aurora PostgreSQL - With containers and API infrastructure

Build a production-ready video content analysis system using scalable AWS cloud infrastructure with [Amazon Bedrock](https://aws.amazon.com/bedrock/), [Amazon Transcribe](https://aws.amazon.com/transcribe/), and [Amazon Aurora PostgreSQL](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/data-api.html) with pgvector extension, [AWS Lambda](https://aws.amazon.com/pm/lambda/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el) and [Amazon API Gateway](https://aws.amazon.com/api-gateway/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el).

![Architecture Diagram](../container-video-embeddings/image/diagram.png)

This notebook demonstrates [Strands Agents](https://strandsagents.com/) integration with **production-grade AWS cloud infrastructure**. Unlike notebook 06 which processes videos locally, this solution uses deployed AWS services for scalable, enterprise-ready video processing.

## 🏗️ Cloud Architecture Components

### **Core Processing Services**
- **[Amazon ECS](https://aws.amazon.com/ecs/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Runs containerized video processing tasks using FFmpeg for frame extraction. Handles compute-intensive operations with auto-scaling capabilities.
- **[AWS Step Functions](https://aws.amazon.com/step-functions/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Orchestrates the video processing workflow, coordinating parallel processing streams for visual and audio content with error handling and retry logic.
- **[Amazon Transcribe](https://aws.amazon.com/transcribe/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Converts speech to text with speaker diarization and timestamp accuracy, creating semantic text chunks while maintaining temporal context.
- **[Amazon Bedrock](https://aws.amazon.com/bedrock/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Generates multimodal embeddings using Titan models for both extracted video frames and transcribed text content.

### **Storage and API Layer**
- **[Amazon Aurora PostgreSQL](https://aws.amazon.com/rds/aurora/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Stores vector embeddings using pgvector extension, supporting combined semantic searches across visual and audio content.
- **[Amazon API Gateway](https://aws.amazon.com/api-gateway/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Provides RESTful endpoints for video upload, processing status checks, and semantic search queries with authentication and rate limiting.
- **[Amazon S3](https://aws.amazon.com/s3/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Stores original videos, extracted frames, transcription results, and processing artifacts with lifecycle management.
- **[AWS Lambda](https://aws.amazon.com/lambda/?trk=4f1e9f0e-7b21-4369-8925-61f67341d27c&sc_channel=el)**: Handles API requests, triggers workflows, and processes results with serverless scaling.

## 🔄 Processing Workflow

The cloud architecture processes videos through these automated steps:

1. **📤 Video Upload**: Video uploaded to S3 bucket triggers Lambda function
2. **⚡ Workflow Orchestration**: Step Functions initiates parallel processing streams
3. **🎬 Visual Pipeline**: ECS tasks extract frames using containerized FFmpeg → Bedrock generates image embeddings
4. **🎤 Audio Pipeline**: Transcribe converts speech to text → Semantic chunking → Bedrock generates text embeddings
5. **📊 Vector Storage**: Lambda functions store all embeddings in Aurora PostgreSQL with pgvector
6. **🌐 API Access**: API Gateway provides endpoints for search queries and status monitoring

## 🤖 Agent Architecture

### 1. **Video Analysis Agent** 
> ⚠️⚠️⚠️⚠️**Infrastructure Requirement**: Deploy the [container-video-embeddings CDK stacks](https://github.com/build-on-aws/langchain-embeddings/tree/main/container-video-embeddings) before running this notebook. ⚠️⚠️⚠️⚠️

- **Purpose**: Processes and searches video content using AWS cloud infrastructure
- **Capabilities**: Analyzes visual frames, transcribed audio, technical content at scale
- **Tools**: `video_embeddings_aws` for cloud-based multimodal video search
- **Use Case**: Enterprise-grade technical content analysis, scalable video processing

![Architecture Diagram](data/agent_videoembedding_cloud.png)


### 2. **Memory-Enhanced Agent**
> **Prerequisites**: ⚠️⚠️⚠️⚠️ Create Amazon Aurora PostgreSQL with this [Amazon CDK Stack](https://github.com/build-on-aws/langchain-embeddings/tree/main/create-aurora-pgvector). Follow steps in [05_create_audio_video_embeddings.ipynb](/05_create_audio_video_embeddings.ipynb) and create an [Amazon S3 verctor bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-buckets-create.html) that will serve as the backend for your vector memory. ⚠️⚠️⚠️⚠️

- **Purpose**: Provides personalized, context-aware video analysis with cloud scalability
- **Capabilities**: Remembers user preferences, learns from interactions, provides tailored responses
- **Tools**: `video_embeddings_aws` + `s3_vector_memory` for persistent user context
- **Use Case**: Personalized learning experiences at scale, adaptive content recommendations

![Architecture Diagram](data/agent_videoembedding_cloud_memory.png)

For detailed explanations of video processing concepts, model options, and S3 memory tools, refer to [notebook 06](06_video_embeddings_with_strands_enhanced.ipynb).

In [1]:
import os
import sys
import boto3
import requests
import json

AWS_REGION = "us-east-1"
VIDEO_PATH = "videos/video.mp4"

sys.path.append('tools')
from video_embeddings_container import video_embeddings_aws

print(f"✅ Configuration loaded")
print(f"🌍 Region: {AWS_REGION}")
print(f"🎬 Video Path: {VIDEO_PATH}")

✅ Configuration loaded
🌍 Region: us-east-1
🎬 Video Path: videos/video.mp4


## 🔧 Cloud Tool Configuration

The `video_embeddings_aws` tool provides cloud-native video processing:

| Parameter | Description | Default |
|-----------|-------------|----------|
| `action` | 'process', 'search', 'list' | 'process' |
| `video_path` | Path to video (local, uploaded to S3) | Required for process |
| `query` | Search query (for search action) | Required for search |
| `region` | AWS region | 'us-east-1' |
| `k` | Number of search results | 10 |
| `max_wait_time` | Maximum wait time for processing (seconds) | 300 |

### Cloud vs Local Processing:
- **Local (notebook 06)**: Direct processing on your machine
- **Cloud (this notebook)**: Scalable processing using AWS services
- **Performance**: Cloud scales automatically based on demand
- **Cost**: Cloud charges only for actual usage
- **Reliability**: Cloud provides high availability and fault tolerance

## 🔍 Infrastructure Validation

> ⚠️⚠️⚠️⚠️**Infrastructure Requirement**: Deploy the [container-video-embeddings CDK stacks](https://github.com/build-on-aws/langchain-embeddings/tree/main/container-video-embeddings) before running this notebook. ⚠️⚠️⚠️⚠️

In [2]:
ssm_client = boto3.client('ssm', region_name=AWS_REGION)
stepfunctions_client = boto3.client('stepfunctions', region_name=AWS_REGION)

try:
    api_endpoint = ssm_client.get_parameter(Name="/videopgvector/api_retrieve", WithDecryption=True)["Parameter"]["Value"]
    bucket_name = ssm_client.get_parameter(Name="/videopgvector/bucket_name", WithDecryption=True)["Parameter"]["Value"]
    state_machine_arn = ssm_client.get_parameter(Name="/videopgvector/state_machine_arn", WithDecryption=True)["Parameter"]["Value"]
    
    print("✅ Infrastructure parameters found:")
    print(f"🔗 API Endpoint: {api_endpoint}")
    print(f"🪣 S3 Bucket: {bucket_name}")
    print(f"⚙️ State Machine: {state_machine_arn.split(':')[-1]}")
    
    test_payload = {"query": "test", "method": "retrieve", "k": 1}
    response = requests.post(api_endpoint, json=test_payload, timeout=10)
    print(f"🌐 API Status: {response.status_code} - {'✅ Connected' if response.status_code == 200 else '❌ Error'}")
    
except Exception as e:
    print(f"❌ Infrastructure check failed: {str(e)}")
    print("💡 Deploy the container-video-embeddings stacks first")

✅ Infrastructure parameters found:
🔗 API Endpoint: https://eq9dp403p9.execute-api.us-east-1.amazonaws.com/prod/retrieve
🪣 S3 Bucket: workflow-stack-videobucket6ed8e1af-vdonyv1wsanm
⚙️ State Machine: wfVideoProcessingWorkflow09A1C915-B52uaM9fDUc6
🌐 API Status: 200 - ✅ Connected


## 🎬 Cloud Video Processing

### 🎥 About the Video Content

This notebook uses the same video from notebooks 05 and 06: **AWS re:Invent 2024 session on "AI self-service support with knowledge retrieval using PostgreSQL"** ([YouTube link](https://www.youtube.com/watch?v=fpi3awGakyg)).

The video covers:
- **Vector databases and embeddings** for AI applications
- **Amazon Aurora PostgreSQL with pgvector** for scalable vector storage
- **RAG (Retrieval Augmented Generation)** implementations
- **Amazon Bedrock Agents** for intelligent customer support
- **Real-world use cases** and technical demonstrations

This content is perfect for testing our video analysis system with technical presentations that include both visual slides and detailed explanations.

In [ ]:
print(f"🎬 Processing video: {VIDEO_PATH}")
print("-" * 60)

result = video_embeddings_aws(
    action="process",
    video_path=VIDEO_PATH,
    region=AWS_REGION,
    max_wait_time=300
)

print(f"\n📊 Processing Result: {result.get('status')}")

if result.get('status') == 'success':
    print(f"✅ Video S3 URI: {result.get('video_s3_uri')}")
    print(f"⏱️ Processing Time: {result.get('processing_time_seconds')}s")
    print(f"🧠 Embeddings Created: {result.get('embeddings_created')}")
elif result.get('status') == 'processing':
    print(f"⏳ {result.get('message')}")
    print(f"📍 Video URI: {result.get('video_s3_uri')}")
else:
    print(f"❌ Error: {result.get('message')}")

## 📊 Understanding the Processing Results

The processing results show:

- **Video S3 URI**: Where the original video is stored
- **Processing Time**: Total time for the cloud workflow
- **Embeddings Created**: Number of vector embeddings generated

This data structure supports both visual and semantic search across your video content using the cloud infrastructure.

## Test 2: Monitor Step Functions

In [ ]:
try:
    response = stepfunctions_client.list_executions(
        stateMachineArn=state_machine_arn,
        maxResults=5
    )
    
    print("📊 Recent Step Functions Executions:")
    print("-" * 50)
    
    for i, execution in enumerate(response['executions'][:3], 1):
        status = execution['status']
        start_time = execution['startDate'].strftime('%Y-%m-%d %H:%M:%S')
        name = execution['name']
        
        status_emoji = {
            'RUNNING': '🔄',
            'SUCCEEDED': '✅',
            'FAILED': '❌',
            'TIMED_OUT': '⏰',
            'ABORTED': '🛑'
        }.get(status, '❓')
        
        print(f"{i}. {status_emoji} {status} - {name}")
        print(f"   Started: {start_time}")
        
        if 'stopDate' in execution:
            stop_time = execution['stopDate'].strftime('%Y-%m-%d %H:%M:%S')
            duration = (execution['stopDate'] - execution['startDate']).total_seconds()
            print(f"   Completed: {stop_time} (Duration: {duration:.1f}s)")
        print()
        
except Exception as e:
    print(f"❌ Error checking Step Functions: {str(e)}")

## Test 3: Search Video Content

In [ ]:
search_queries = [
    "what is aurora",
    "people talking",
    "technology",
    "presentation"
]

print("🔍 Performing semantic search queries...")
print("=" * 60)

for i, query in enumerate(search_queries, 1):
    print(f"\n{i}. Query: '{query}'")
    print("-" * 40)
    
    search_result = video_embeddings_aws(
        action="search",
        query=query,
        region=AWS_REGION,
        k=5
    )
    
    if search_result.get('status') == 'success':
        results = search_result.get('results', [])
        print(f"✅ Found {len(results)} results")
        
        for j, result in enumerate(results[:3], 1):
            similarity = result.get('similarity_score', 0)
            content_type = result.get('content_type', 'unknown')
            source = result.get('source', 'N/A')
            preview = result.get('content_preview', '')
            
            print(f"\n   {j}. Similarity: {similarity:.3f} | Type: {content_type}")
            print(f"      Source: {source}")
            if preview:
                print(f"      Preview: {preview}")
    else:
        print(f"❌ Search failed: {search_result.get('message')}")
    
    print()

## 🔍 Search Results Explained

The search results demonstrate multimodal embeddings:

- **Type**: 'text' (from transcription) or 'image' (from visual frames)
- **Similarity Score**: Higher scores (closer to 1.0) indicate better matches
- **Source**: File path or timestamp reference
- **Content**: Preview of matched text or frame description

This lets you find specific moments in videos using natural language queries, whether the content appears visually or is spoken in the audio.

## Test 4: List Frames

In [ ]:
print(f"📋 Listing all processed Frames")
print("-" * 50)

list_result = video_embeddings_aws(
    action="list",
    region=AWS_REGION,
)

print(f"\n📊 List Result:")
print(f"Status: {list_result.get('status')}")

if list_result.get('status') == 'success':
    videos = list_result.get('videos', [])
    print(f"📹 Total Frames: {list_result.get('total_videos')}")
    print(f"🧠 Total Embeddings: {list_result.get('total_embeddings')}")
    
    for i, video in enumerate(videos[:5], 1):
        print(f"\n{i}. Frames:")
        print(f"   📍 Source: {video.get('source_url')}")
        print(f"   🎭 Type: {video.get('content_type')}")
        print(f"   🧠 Embeddings: {video.get('embedding_count')}")
else:
    print(f"❌ Error: {list_result.get('message')}")

## 🤖 Create agents with Strands Agents

Create intelligent agents that use the scalable cloud architecture. For detailed model configuration options, see [notebook 06](06_video_embeddings_with_strands_enhanced.ipynb).

## 🎯 Model Configuration Options

Strands supports multiple model configuration approaches:

### Option 1: Default Configuration
```python
from strands import Agent
agent = Agent()  # Uses Claude 4 Sonnet with Amazon Bedrock by default
```

### Option 2: Specify Model ID
```python
agent = Agent(model="anthropic.claude-sonnet-4-20250514-v1:0")
```

### Option 3: BedrockModel (Recommended)
```python
from strands.models import BedrockModel

model = BedrockModel(
    model_id="anthropic.claude-sonnet-4-20250514-v1:0",
    temperature=0.3,
    top_p=0.8
)
agent = Agent(model=model)
```

### Option 4: Anthropic Direct
```python
from strands.models.anthropic import AnthropicModel

model = AnthropicModel(
    model_id="claude-sonnet-4-20250514",
    max_tokens=1028,
    params={"temperature": 0.7}
)
```

You can also use other model providers:
- [LiteLLM](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/litellm/)
- [llama.cpp](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/llamacpp/)
- [Llama API](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/llamaapi/)
- [Mistral AI](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/mistral/)
- [Ollama](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/ollama/)
- [OpenAI](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/openai/)
- [Cohere](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/openai/)

**BedrockModel Benefits:**
- Native AWS integration
- Guardrails support
- Prompt caching capabilities

In [3]:
from strands.models import BedrockModel
from strands import Agent
from video_image_display import display_video_images

model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0")

CLOUD_SYSTEM_PROMPT = """You are a video processing AI assistant.

Available actions:
- process: Upload and process videos 
- search: Search video content using semantic similarity
- list: List all processed videos

Use video_embeddings_aws for all cloud video operations.
Use display_video_images to show search results.
"""

cloud_video_agent = Agent(
    model=model,
    tools=[video_embeddings_aws, display_video_images],
    system_prompt=CLOUD_SYSTEM_PROMPT
)

print("✅ Cloud-native Strands Video Agent created!")

✅ Cloud-native Strands Video Agent created!


## 🧠 Memory-Enhanced Strands Agent

Create an agent with S3 memory capabilities for personalized cloud interactions. For detailed S3 memory tool explanation, see [notebook 06](06_video_embeddings_with_strands_enhanced.ipynb).

In [4]:
from s3_memory import s3_vector_memory

# S3 Vectors Configuration
# S3 Vectors Configuration
os.environ['VECTOR_BUCKET_NAME'] = 'YOUR-S3-BUCKET'  # Your S3 Vector bucket
os.environ['VECTOR_INDEX_NAME'] = 'YOUR-VECTOR-INDEX'        # Your vector index
os.environ['AWS_REGION'] = 'us-east-1'                       # AWS region
os.environ['EMBEDDING_MODEL'] = 'amazon.titan-embed-text-v2:0' # Bedrock embedding model

cloud_memory_agent = Agent(
    model=model,
    tools=[video_embeddings_aws, display_video_images, s3_vector_memory],
    system_prompt=CLOUD_SYSTEM_PROMPT
)

print("✅ Memory-enhanced agent created!")

✅ Memory-enhanced agent created!


## 🧪 Test Agents

Test the agents with intelligent queries:

In [5]:
print("🧪 Testing Cloud Video Agent")
print("=" * 50)

response1 = cloud_video_agent(
    "Search for content about 'aurora database scalability' using the cloud infrastructure. Return the top 5 results and explain what you found."
)
print(f"Cloud Agent Response: {response1.message}")

🧪 Testing Cloud Video Agent
I'll search for content about 'aurora database scalability' in the processed videos and return the top 5 results.
Tool #1: video_embeddings_aws
Let me try the search again with a default AWS region:
Tool #2: video_embeddings_aws
🔍 Searching for: 'aurora database scalability'
✅ Found 5 matching results
Great! I found 5 results related to "aurora database scalability". Let me also display the images from these search results to provide you with visual context:
Tool #3: display_video_images
## Search Results for "Aurora Database Scalability"

I successfully found 5 relevant results from the video content using semantic search. Here's what I discovered:

### **Summary of Findings:**

The search returned content from two main video sources discussing database scalability, particularly focusing on AWS services and PostgreSQL compatibility. Here's a breakdown of the top 5 results:

**1. Image Content (Highest Similarity: 0.513)**
- **Source:** langchain_test_user_2

In [6]:
response2 = cloud_video_agent(
    "List all processed videos in the cloud storage and provide a summary of available content with processing statistics."
)
print(f"\nCloud Storage Summary: {response2.message}")

I'll list all the processed videos in the cloud storage and provide you with a comprehensive summary of the available content and processing statistics.
Tool #4: video_embeddings_aws
📋 Listing all processed videos
✅ Found 37 processed videos
## 📊 Video Processing Summary & Statistics

### **🎬 Available Content Overview**

I found **37 processed content items** stored in the cloud, consisting of **2 main video sources** with comprehensive text and visual analysis:

---

### **📹 Main Video Sources**

#### **1. dev315.mp4**
- **Location:** `s3://workflow-stack-videobucket6ed8e1af-vdonyv1wsanm/video/dev315.mp4`
- **Text Embeddings:** 15 segments
- **Image Frames:** 35 extracted key frames
- **Content Type:** Multi-speaker technical content
- **Key Timestamps:** Frames from 34s to 2097s (35+ minutes of content)

#### **2. langchain_test_user_2/video.mp4**
- **Location:** `s3://workflow-stack-videobucket6ed8e1af-vdonyv1wsanm/videos/langchain_test_user_2/video.mp4`
- **Text Embeddings:** 50 s

In [ ]:
print("\n3. Processing video (if available):")
response3 = cloud_video_agent(
    f"If the video file {VIDEO_PATH} exists, process it. Wait up to 2 minutes and report results."
)
print(f"Agent Response: {response3.message}")

In [7]:
USER_ID = "cloud_test_user"

response3 = cloud_memory_agent(f"""For user {USER_ID}:
1. Store my interest in cloud architecture and scalable video processing
2. Search the video content for 'production deployment' and 'scalability'
3. Remember key insights about cloud-native video processing
4. Provide a personalized summary based on my cloud architecture interests""")

print(f"\nPersonalized Cloud Analysis: {response3.message}")

I'll help you with that comprehensive workflow. Let me start by storing your interests, then search for video content, and finally provide a personalized summary.
Tool #1: s3_vector_memory

Tool #2: video_embeddings_aws
Let me try the search with a region specified:
Tool #3: video_embeddings_aws
🔍 Searching for: 'production deployment'
✅ Found 5 matching results
Now let me search for scalability content:
Tool #4: video_embeddings_aws
🔍 Searching for: 'scalability'
✅ Found 5 matching results
Now let me store the key insights about cloud-native video processing based on the search results:
Tool #5: s3_vector_memory
Now let me retrieve your stored interests to create a personalized summary:
Tool #6: s3_vector_memory
## Personalized Summary Based on Your Cloud Architecture Interests

Based on your interests in **cloud architecture and scalable video processing**, here's what I discovered from the video content analysis:

### 🏗️ **Production Deployment Insights**
- **Deployment Efficiency**

In [12]:
response4 = cloud_memory_agent(f"I am {USER_ID}, How many videos are there in the database?")
print(f"\nPersonalized Cloud Analysis: {response4.message}")


Tool #7: video_embeddings_aws
📋 Listing all processed videos
✅ Found 37 processed videos
Based on the database query, here's the current video content status:

## 📊 Video Database Summary

**Total Videos in Database: 2 video files**

### 📹 Video Files:
1. **`dev315.mp4`** - 15 text embeddings
2. **`langchain_test_user_2/video.mp4`** - 50 text embeddings

### 🖼️ Video Frames:
- **35 extracted image frames** from `dev315.mp4` (1 embedding each)

### 📈 Database Statistics:
- **Total Content Items**: 37 (2 videos + 35 image frames)
- **Total Embeddings**: 100 
- **Storage Backend**: Aurora PostgreSQL with pgvector
- **Embedding Model**: Amazon Titan Embed

### 🔍 Content Breakdown:
- **Text Content**: 65 embeddings (from video transcripts/audio)
- **Image Content**: 35 embeddings (from extracted video frames)
- **Searchable Segments**: All content is semantically searchable

The system has processed both videos completely, extracting both audio/text content and key visual frames for compre

In [13]:
response5 = cloud_memory_agent(f"I am {USER_ID}, Give me a summary of the content of each video")
print(f"\nPersonalized Cloud Analysis: {response5.message}")

Let me search for content from each video to provide you with comprehensive summaries.
Tool #8: video_embeddings_aws
🔍 Searching for: 'dev315'
✅ Found 10 matching results
Let me get more comprehensive content by searching for general topics from each video:
Tool #9: video_embeddings_aws
🔍 Searching for: 'AWS cloud infrastructure deployment database'
✅ Found 15 matching results

Tool #10: video_embeddings_aws
🔍 Searching for: 'knowledge base AI vector database application'
✅ Found 10 matching results
Now let me store this comprehensive summary in your memory:
Tool #11: s3_vector_memory
## 📹 Comprehensive Video Content Summary

Based on my analysis of both videos in the database, here are the detailed summaries:

### Video 1: `dev315.mp4` (15 text embeddings)
### Video 2: `langchain_test_user_2/video.mp4` (50 text embeddings)

**Note**: These appear to be the same content with different processing granularity.

---

## 🎯 **Main Topics Covered**

### 1. **AWS Infrastructure as Code with C

## 🚀 Production Deployment Considerations

### **Scaling Configuration**
- **ECS Auto Scaling**: Configure based on CPU/memory utilization
- **Aurora Scaling**: Use Aurora Serverless v2 for automatic scaling
- **API Gateway**: Configure throttling and caching for optimal performance

### **Cost Optimization**
- **S3 Intelligent Tiering**: Automatically move infrequently accessed videos to cheaper storage
- **Spot Instances**: Use for ECS tasks to reduce compute costs
- **Reserved Capacity**: Purchase reserved capacity for predictable workloads

### **Security Best Practices**
- **IAM Roles**: Use least privilege access for all services
- **VPC Configuration**: Deploy in private subnets with NAT Gateway
- **Encryption**: Use encryption at rest and in transit for all data

### **Monitoring and Alerting**
- **CloudWatch Dashboards**: Monitor processing times, error rates, costs
- **X-Ray Tracing**: Track request flows through the system
- **SNS Notifications**: Alert on processing failures or cost thresholds

## 🔗 Next Steps

- Explore the [container-video-embeddings repository](https://github.com/build-on-aws/langchain-embeddings/tree/main/container-video-embeddings) for deployment details
- Test with your own video content at scale
- Integrate the API endpoints into your applications
- Set up monitoring and alerting for production workloads
- Configure auto-scaling policies based on your usage patterns